In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from xgboost import XGBClassifier
import joblib
import rasterio
from skimage.util.shape import view_as_windows
from joblib import Parallel, delayed
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load the data and train the models
df = pd.read_csv("preprocessed_data.csv")
X = df.drop(columns=['label'])
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=41, stratify=y)
rf_model = RandomForestClassifier(n_estimators=100)
svm_model = SVC(kernel='rbf')
clf = xgb.XGBClassifier(objective="multi:softprob", random_state=42) 

rf_model.fit(X_train, y_train)
svm_model.fit(X_train, y_train)
clf.fit(X_train, y_train-1)

y_pred = rf_model.predict(X_test)
print("ACCURACY OF Random Forest MODEL:", metrics.accuracy_score(y_test, y_pred))

y_pred = svm_model.predict(X_test)
print("ACCURACY OF SVM MODEL:", metrics.accuracy_score(y_test, y_pred))

y_pred =clf.predict(X_test)
print("ACCURACY OF XGBoost MODEL:", metrics.accuracy_score(y_test-1, y_pred))


X = df.iloc[:, :-1].values  # Features
y = df.iloc[:, -1].values   # Labels

# Normalize input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define neural network architecture
nn_model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(6, activation='softmax')# Adjust num_classes according to your dataset
])

# Compile the model
nn_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = nn_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)
y_pred = nn_model.predict(X_test)
y_pred_p = np.argmax(y_pred, axis=1)
y_pred_p = y_pred_p.reshape(-1, 1)

test_loss, test_acc = nn_model.evaluate(X_test, y_test)
print(f'ACCURACY OF Neural Network MODEL: {test_acc:.4f}')

joblib.dump(rf_model, 'random_forest_model.pkl')
joblib.dump(svm_model, 'svm_model.pkl')
joblib.dump(clf, 'xgb_model.pkl')
nn_model.save('nn_model.keras')

ACCURACY OF Random Forest MODEL: 0.83841059602649
ACCURACY OF SVM MODEL: 0.7019867549668874
ACCURACY OF XGBoost MODEL: 0.833112582781457


C:\Users\HP\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.2085 - loss: 1.7337 - val_accuracy: 0.4491 - val_loss: 1.4454
Epoch 2/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4602 - loss: 1.4055 - val_accuracy: 0.6179 - val_loss: 1.1458
Epoch 3/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5639 - loss: 1.1566 - val_accuracy: 0.6600 - val_loss: 0.9520
Epoch 4/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6152 - loss: 0.9914 - val_accuracy: 0.6749 - val_loss: 0.8653
Epoch 5/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6546 - loss: 0.9199 - val_accuracy: 0.6774 - val_loss: 0.8193
Epoch 6/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6529 - loss: 0.8743 - val_accuracy: 0.6849 - val_loss: 0.8237
Epoch 7/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6685 - loss: 0.8348 - val_accuracy: 0.6824 - val_loss: 0.7703
Epoch 8/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6763 - loss: 0.8152 - val_accuracy: 0.7022 - val_loss